In [23]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import cProfile

from itertools import islice, takewhile, chain
from functools import reduce
from typing import Optional
import datetime as dt
from dataclasses import asdict, fields
from importlib import reload

from geopy.distance import distance
from shapely.geometry import Point, LineString
import shapely.geometry as sg
import geopandas as gpd

from typing import List
import ipyleaflet as lf

pd.set_option('display.max_rows', 1000000000)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 160)

In [2]:
import busboy.model as m
import busboy.geo as geo
import busboy.database as db
import busboy.prediction as prediction
import busboy.prediction.pandas
import busboy.prediction.sklearn
import busboy.map.map as bmap
import busboy.apis as api
import busboy.util as util
import busboy.util.notebooks as notebook

/Users/Noel/.local/share/virtualenvs/Busboy-8t8akAoa/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [50]:
reload(util)
reload(geo)
reload(m)
reload(db)
reload(prediction)
reload(busboy.prediction.pandas)
reload(busboy.prediction.sklearn)
reload(bmap)
reload(api)
reload(notebook)

<module 'busboy.util.notebooks' from '/Users/Noel/Developer/Projects/Busboy/busboy/util/notebooks.py'>

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor

from sklearn.model_selection import cross_val_score

# Class, for use in pipelines, to select certain columns from a DataFrame and convert to a numpy array
# From A. Geron: Hands-On Machine Learning with Scikit-Learn & TensorFlow, O'Reilly, 2017
# Modified by Derek Bridge to allow for casting in the same ways as pandas.DatFrame.astype
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names, dtype=None):
        self.attribute_names = attribute_names
        self.dtype = dtype
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_selected = X[self.attribute_names]
        if self.dtype:
            return X_selected.astype(self.dtype).values
        return X_selected.values

In [51]:
rbn = db.routes_by_name()
route = rbn["220"]
entries = db.snapshots(
    r=route.id, 
#     date_span=(dt.date(2019, 3, 2), dt.date(2019, 3, 5))
    d=dt.date(2019, 3, 2)
)
stops_by_name = db.stops_by_name()
timetables = [t.value for t in db.timetables(route.id) if isinstance(t, util.Right)]
timetable_variants = {t for timetable in timetables for t in timetable.variants}

In [52]:
len(entries)

116699

In [53]:
entries_by_vehicle = util.dict_collect_list(entries, lambda e: e.vehicle)
vehicles_by_entry_count = [v for (v, es) in sorted(entries_by_vehicle.items(), key = lambda t: len(t[1]), reverse=True)]

In [54]:
route_sections = {
    variant: list(prediction.route_sections(variant.stops)) 
    for variant in timetable_variants
}

In [55]:
def get_all_journeys(entries_by_vehicle, timetable_variants):
    return [
        prediction.sklearn.journeys(entries_by_vehicle[vehicle], timetable_variants, route_sections) 
        for vehicle in vehicles_by_entry_count
        if vehicle.raw is not None
    ]

In [59]:
cProfile.run("get_all_journeys(entries_by_vehicle, timetable_variants)")

         521540430 function calls (325432262 primitive calls) in 176.846 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    35340    0.011    0.000    0.041    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
      252    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:416(parent)
        1    0.000    0.000  220.043  220.043 <ipython-input-55-a5a33d6fdfe2>:1(get_all_journeys)
        1    0.031    0.031  220.043  220.043 <ipython-input-55-a5a33d6fdfe2>:3(<listcomp>)
        1    0.001    0.001  220.044  220.044 <string>:1(<module>)
       27    0.000    0.000    0.000    0.000 <string>:1(__hash__)
    51318    0.005    0.000    0.005    0.000 <string>:1(__init__)
  2062216    2.634    0.000   46.315    0.000 __init__.py:119(cached_contains)
 25372534   16.177    0.000   66.306    0.000 __init__.py:127(contains)
    33473    0.027    0.000   11.254    0.000 __init__.py:142(dict_collect_set)
    794

In [60]:
prediction.cached_contains.cache_info()

CacheInfo(hits=32991044, misses=4467800, maxsize=None, currsize=4467800)

In [61]:
all_journeys = get_all_journeys(entries_by_vehicle, timetable_variants)

In [62]:
big_dfs = prediction.sklearn.join_journeys(all_journeys)
day_df = max(big_dfs.items(), key=lambda t: len(t[0].stops))
day_df[1]

,Ovens (Grange Road Terminus) [arrival],Ovens (Grange Road Terminus) [departure],Ovens (Opp Grange Manor) [arrival],Ovens (Opp Grange Manor) [departure],Killumney Road (Kilumney Cross) [arrival],Killumney Road (Kilumney Cross) [departure],Ovens (EMC Terminus) [arrival],Ovens (EMC Terminus) [departure],Ballincollig West (Classes Lake) [arrival],Ballincollig West (Classes Lake) [departure],Ballincollig West (Opp Aylsbury Estate) [arrival],Ballincollig West (Opp Aylsbury Estate) [departure],Ballincollig West (Opp Old Quarry) [arrival],Ballincollig West (Opp Old Quarry) [departure],Ballincollig West (Opp Coolroe Heights) [arrival],Ballincollig West (Opp Coolroe Heights) [departure],Ballincollig West (Op White Horse Bar) [arrival],Ballincollig West (Op White Horse Bar) [departure],Ballincollig (Opp Jctn Barrys Road) [arrival],Ballincollig (Opp Jctn Barrys Road) [departure],Ballincollig (Shopping Centre) [arrival],Ballincollig (Shopping Centre) [departure],Ballincollig Town Centre (Garda St) [arrival],Ballincollig Town Centre (Garda St) [departure],Ballincollig (East Gate) [arrival],Ballincollig (East Gate) [departure],Ballincollig (Opp Rosewood Est) [arrival],Ballincollig (Opp Rosewood Est) [departure],Model Farm Rd (Guide Dog Centre) [arrival],Model Farm Rd (Guide Dog Centre) [departure],Model Farm Rd (Church Cross East) [arrival],Model Farm Rd (Church Cross East) [departure],Model Farm Rd (Inchaggin Eastbound) [arrival],Model Farm Rd (Inchaggin Eastbound) [departure],Model Farm Road (Eden Hall) [arrival],Model Farm Road (Eden Hall) [departure],Model Farm Rd (Opp Ultralase Ireland) [arrival],Model Farm Rd (Opp Ultralase Ireland) [departure],Model Farm Rd (IDA Technology Park) [arrival],Model Farm Rd (IDA Technology Park) [departure],Model Farm Rd (Dept of Agriculture) [arrival],Model Farm Rd (Dept of Agriculture) [departure],Model Farm Road (Farranlea Park) [arrival],Model Farm Road (Farranlea Park) [departure],Model Farm Rd (Bishopstown Park) [arrival],Model Farm Rd (Bishopstown Park) [departure],Model Farm Rd (Dennehy's Cross Jctn) [arrival],Model Farm Rd (Dennehy's Cross Jctn) [departure],Dennehy's Cross (Opp Cork Farm Ctr) [arrival],Dennehy's Cross (Opp Cork Farm Ctr) [departure],Victoria Cross (Victoria Lodge) [arrival],Victoria Cross (Victoria Lodge) [departure],Western Rd (Opp UCC Western Gateway) [arrival],Western Rd (Opp UCC Western Gateway) [departure],Western Rd (Opp UCC Castlewhite) [arrival],Western Rd (Opp UCC Castlewhite) [departure],Western Road (Gaol Crossl) [arrival],Western Road (Gaol Crossl) [departure],Western Road (Opp University College Gat [arrival],Western Road (Opp University College Gat [departure],Mardyke Walk (St. Joseph's School) [arrival],Mardyke Walk (St. Joseph's School) [departure],Mardyke (Presentation College) [arrival],Mardyke (Presentation College) [departure],Sheares Street (Mercy Hospital) [arrival],Sheares Street (Mercy Hospital) [departure],Grand Parade (Caseys Furniture) [arrival],Grand Parade (Caseys Furniture) [departure],South Mall (Opp Cork Passport Office) [arrival],South Mall (Opp Cork Passport Office) [departure],Cork City Hall [arrival],Cork City Hall [departure],Southern Rd (Opp Owl Printers) [arrival],Southern Rd (Opp Owl Printers) [departure],Douglas Road (Opp St Finbarrs Hospital) [arrival],Douglas Road (Opp St Finbarrs Hospital) [departure],Douglas Road (Glengesh Bellair) [arrival],Douglas Road (Glengesh Bellair) [departure],Douglas Road (Cross Douglas Rd Jctn) [arrival],Douglas Road (Cross Douglas Rd Jctn) [departure],Douglas Road (Before Woolharra Park) [arrival],Douglas Road (Before Woolharra Park) [departure],Douglas Road (Ardfallen Shopping Mall) [arrival],Douglas Road (Ardfallen Shopping Mall) [departure],Douglas Road (Endsleigh Estate) [arrival],Douglas Road (Endsleigh Estate) [departure],Douglas Road (Clermont Ave) [arrival],Douglas Road (Clermont Ave) [departure],Douglas East Village (Opp Tramway Tce) [arrival],Douglas East Village (Opp Tramway Tce) [departure],Maryb

In [20]:
last = f"{day_df[0].stops[5].name} [departure]"
target = f"{day_df[0].stops[6].name} [arrival]"
df = day_df[1]
df = df[pd.notnull(df[target]) & pd.notnull(df[last])]
travel_times = prediction.pandas.travel_times(
    df, [], 
    last, 
    target)

In [21]:
y = travel_times.astype("int64") / 1_000_000_000
y

array([ 30.004392,   0.      ,  20.015994,   0.      ,  20.01578 ,
        20.001199,   0.      , 110.086088,  20.019048,  10.008478,
        40.016994,  20.011626,  20.017602,   0.      ,  40.031647,
        40.011332,  20.011683,  20.015321,  40.020732,  40.02117 ,
        40.035162,  20.0162  ,  30.014912,  20.015725,  20.017682,
        40.025228,   0.      ,  10.008014,  20.015809,  40.024694,
        20.012141,  20.013693,  20.015234,  20.016373,  40.037717,
        20.017223,  20.019086,  60.045201,  20.016668,  80.073725,
        20.017566,  40.02071 ,  30.011126, 120.06057 ,   0.      ,
         0.      ,  10.000204,  20.017029,  60.034554,  40.022258,
         0.      ,   0.      ,  10.008305,  40.033811,  40.021201,
        20.017389,   0.      ,  60.037838,  10.007748,   0.      ,
        40.031967,  20.01309 ,  20.01646 ,  20.017098,   0.      ,
        10.002656,  40.032717,  40.031905,   0.      ,  40.017786,
        20.018424,  20.016604,  20.004446,  60.022328,  20.016

In [22]:
pipeline = Pipeline([
    ("selector", DataFrameSelector(list(df))),
    ("estimator", DummyRegressor(strategy="median")),
])
mean_error = np.mean(cross_val_score(pipeline, df, y, scoring="neg_mean_absolute_error", cv=4))
-mean_error

15.759215842697149

In [41]:
import datetime
time1 = datetime.datetime(2019, 2, 18, 11, 45, 12)
time2 = datetime.datetime(2019, 3, 1, 10, 22, 18)
test_times1 = np.array([time1], dtype="datetime64[us]")
test_times2 = np.array([time2], dtype="datetime64[us]")
a = test_times2 - test_times1
mean = a.mean()
average = np.average(a)
print(f"mean: {mean.astype('int64')}", f"average: {average}")

ValueError: Could not convert object to NumPy timedelta

In [12]:
a.mean()
np.average(a)

ValueError: Could not convert object to NumPy timedelta

In [180]:
avg = a.mean(0)
print(avg.size)
print(avg)
size_ratio = a.size/avg.size
type(size_ratio)
avg.dtype.type(int(size_ratio))
avg
np.median(a)

1
1160500000 microseconds


numpy.timedelta64(1160500000,'us')

In [53]:
to_time = lambda d: d.time().isoformat()
for variant, these_stop_times in times.items():
    print(f"Variant: {variant}")
    print(f"{len(these_stop_times)} journeys")
    for trip_number, trip_times in enumerate(these_stop_times):
        for stop_number, stop_time in enumerate(trip_times):
            to_time = lambda d: d.time().isoformat()
            try:
                stop_name = variant.stops[stop_number].name
            except IndexError:
                stop_name = "(IndexError)"
            print(f"- {trip_number}, {stop_number:2}, {stop_name:40}"
                  f" {stop_time.last_before.map(to_time).or_else(''):15}, {stop_time.first_after.map(to_time).or_else(''):15}")

Variant: (route: 201, start: Boherboy Rd (Opp Scoil Mhuire Banrion), end: CUH (Bishopstown Rd))
21 journeys
- 0,  0, Boherboy Rd (Opp Scoil Mhuire Banrion)                  , 07:19:32.536890
- 0,  1, Boherboy Road (Lotabeg Estate)           07:18:52.525808, 07:20:32.577308
- 0,  2, North Ring Road (Opp Mayfield Supermarke 07:20:12.559419, 07:21:32.628519
- 0,  3, North Ring Road (Lagan Grove)            07:21:32.628519, 07:22:52.652098
- 0,  4, North Ring Rd (Glencree Crescent)        07:22:52.652098, 07:23:32.658942
- 0,  5, North Ring Rd (Corrib Lawn)              07:23:32.658942, 07:24:32.690832
- 0,  6, North Ring Rd (Boyne Crescent)           07:24:32.690832, 07:25:12.698864
- 0,  7, North Ring Rd (Opp Riverview Estate)     07:25:52.734574, 07:27:32.789463
- 0,  8, Old Commons Rd (Opp Topaz Service St)    07:29:32.858933, 07:30:52.904094
- 0,  9, Farranferris Ave (Pophams Rd Junction)   07:30:52.904094, 07:32:32.993772
- 0, 10, Pophams Rd (Opposite Community Centre)   07:31:52.958

In [50]:
themap = bmap.Map()
themap.map

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [51]:
notebook.show_timetables(themap, timetables)

In [56]:
themap.map.add_control(lf.LayersControl())

In [52]:
[(v, len(entries_by_vehicle[v])) for v in vehicles_by_entry_count]

[(VehicleId(raw='7338674957838189376'), 7885), (VehicleId(raw=None), 658)]